<a href="https://colab.research.google.com/github/Kaushalsurana/SIH_hackathon/blob/main/Xception_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import Javascript

def disable_colab_idle_timeout():
    js_code = '''
    function ClickConnect(){
        console.log("Working");
        document.querySelector("colab-toolbar-button#connect").click()
    }
    setInterval(ClickConnect,60000)
    '''
    display(Javascript(js_code))

disable_colab_idle_timeout()

In [ ]:
!pip install opendatasets
!pip install pandas
import opendatasets as od
import pandas

# od.download("https://www.kaggle.com/datasets/ascanipek/skin-diseases")

In [ ]:
# from keras.layers import Input, Lambda, Dense, Flatten
# from keras.models import Model
# from keras.applications.vgg16 import VGG16
# from keras.applications.vgg16 import preprocess_input
# from keras.preprocessing import image
# from keras.preprocessing.image import ImageDataGenerator
# from keras.models import Sequential
# import numpy as np
# from glob import glob
# import matplotlib.pyplot as plt

# # re-size all the images to this
# IMAGE_SIZE = [224, 224]



from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

# re-size all the images to this
IMAGE_SIZE = [224, 224]


In [ ]:
train = r"C:\Users\vibhor\Desktop\datastes\kaggle\train"
test = r"C:\Users\vibhor\Desktop\datastes\kaggle\test"
val = r"C:\Users\vibhor\Desktop\datastes\kaggle\val"

In [ ]:
from tensorflow.keras.applications.xception import Xception
ress = Xception(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

#for layer in ress.layers:
  #layer.trainable = False

In [ ]:
folders = glob(r"C:\Users\vibhor\Desktop\datastes\kaggle\train\*")
len(folders)

In [ ]:
import tensorflow as tf
 # useful for getting number of classes

x = Flatten()(ress.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=ress.input, outputs=prediction)

# view the structure of the model
model.summary()

# tell the model what cost and optimization method to use
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.0007,momentum=0.9),
  loss='categorical_crossentropy',
  metrics=['accuracy']
)

In [ ]:
# from keras.preprocessing.image import ImageDataGenerator

# train_datagen = ImageDataGenerator(rescale = 1./255,
#                                    shear_range = 0.2,
#                                    zoom_range = 0.2,
#                                    horizontal_flip = True)

# test_datagen = ImageDataGenerator(rescale = 1./255)

# training_set = train_datagen.flow_from_directory('/content/skin-diseases/kaggle/train',
#                                                  target_size = (224, 224),
#                                                  batch_size = 32,
#                                                  class_mode = 'categorical')

# test_set = test_datagen.flow_from_directory('/content/skin-diseases/kaggle/test',
#                                             target_size = (224, 224),
#                                             batch_size = 32,
#                                             class_mode = 'categorical')











from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Initialize the ImageDataGenerator for training and testing sets
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1.0/255)

# Update the dataset paths to your local paths
training_set = train_datagen.flow_from_directory(r"C:\Users\vibhor\Desktop\datastes\kaggle\train",
                                                 target_size=(224, 224),
                                                 batch_size=32,
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory(r"C:\Users\vibhor\Desktop\datastes\kaggle\test",
                                            target_size=(224, 224),
                                            batch_size=32,
                                            class_mode='categorical')


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, verbose=1, mode='auto', restore_best_weights=True)
model_checkpoint = ModelCheckpoint('skin_disease_model.keras', save_best_only=True)



In [ ]:
r = model.fit(
    training_set,
    validation_data=test_set,
    epochs=25,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set),
    callbacks=[early, model_checkpoint]
)


In [ ]:
model.save('final_skin_disease_model.h5')

In [ ]:
import matplotlib.pyplot as plt

# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('final_skin_disease_model.h5')

from keras.preprocessing import image
import numpy as np

# Load and preprocess the test image

test_image = image.load_img('/content/skin-diseases/kaggle/test/1. Enfeksiyonel/3_56.jpg', target_size=(224, 224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

# Make predictions using the loaded model
predictions =loaded_model.predict(test_image)
print(predictions)
# Get the index of the predicted class
predicted_class_index = np.argmax(predictions)

# Print the predicted class index
print("Predicted Class Index:", predicted_class_index)
img = image.load_img('/content/skin-diseases/kaggle/test/1. Enfeksiyonel/3_56.jpg', target_size=(224, 224))
plt.imshow(img)
plt.axis('off')
plt.show()
